# Multilevel QPQ algorithm


- 1: Estimate the preference $\theta_{c,i}(r)$ \\ %(importance of her transaction to appear in position $r$)
- 2: Calculate the normalized preference $\bar{\theta}_{c,i}(r)=\mathit{PIT}_{c,i}(\theta_{c,i}(r))$ 
- 3:  Declare a value $\dot{\theta}_{c,i}(r)$.
- 4: Wait to receive the declared preferences from all players
- 5: For all $j\in\mathcal{N}_c$ do
    - 6: if not GoF$\_$Test($\dot{\theta}_{c,j}(r)$, $\mathit{CHistory}_{c,j}$) then
        - 7: $\ddot{\theta}_{c,j}(r) \leftarrow \hat{\theta}_{c,j}(r)$, where $\hat{\theta}_{c,j}(r) := \textrm{Pseudorandom}(\{\dot{\theta}_{c,m}(r)\}_{m\in\mathcal N_c \backslash \{j\} }$)
    - 8: else
        - 9: $\ddot{\theta}_{c,j}(r)$ $\leftarrow$ $\dot{\theta}_{c,j}(r)$
    - 10: end if
    - 11: $\mathit{CHistory}_{c,j}$ $\leftarrow$ $\mathit{CHistory}_{c,j} \cup \{\ddot{\theta}_{c,j}(r)\}$
- 12: end for
- 13: Let $d=\arg \max_{j\in \mathcal{N}_c} \{ \ddot{\theta}_{c,j}(r) \}$ 
- 14: if $d=i$ then // Player $(c,i)$ is the winner of cluster $c$;
    - 15: Declare value $\ddot{\theta}_{c,i}(r)$ to all the players. 
- 16: end if
- 17: Wait to receive the value $\ddot{\theta}_{w,b_w}(r)$ from the winners of the rest of the clusters.
- 18: Let $\mathcal N^{(1)}=\{(1,b_1),\dots,(k,b_k)\}$ be the set of winners
- 19: For all $(w,b_w) \in\mathcal{N}^{(1)}$ do
    - 20: if not GoF$\_$Test($\ddot{\theta}_{w,b_w}(r)$, $\mathit{SCHistory}_{w}$) then
        - 21: $\dddot{\theta}_{w,b_w}(r) \leftarrow \hat{\hat{\theta}}_{w,b_w}(r)$, where $\hat{\hat{\theta}}_{w,b_w}(r) := \textrm{Pseudorandom}(\{\ddot{\theta}_{x,b_x}(r)\}_{(x,b_x) \in \mathcal N^{(1)}\backslash \{(w,b_w)\} })$
    - 22: else
        - 23: $\dddot{\theta}_{w,b_w}(r)$ $\leftarrow$ $\ddot{\theta}_{w,b_w}(r)$\\ 
    - 24: end if 
    - 25: $\mathit{SCHistory}_{w}$ $\leftarrow$ $\mathit{SCHistory}_{w} \cup \{\dddot{\theta}_{w,b_w}(r)\}$ 
- 26: end for 
- 27: Let $d=\arg \max_{(w,b_w)\in\mathcal{N}^{(1)}} \{ \dddot{\theta}_{w,b_w}(r) \}$
- 28: if $d=(c,i)$ then
    - 29: Player $i$ gets resource $r$
- 30: end if

In [ ]:
# Values is a matrix (N x r) or (number of playes x number of rounds)
# Level 0 is base level 
# Level 1 is cluster level
def QPQ2Level(declaredVals, trueVals, K, histlenL0, maxpvalL0, histlenL1, maxpvalL1, KSTest=True, debug=True):
    dims = declaredVals.shape
    N = dims[0] # number of players
    R = dims[1] # number of rounds
    playersPerCluster = int(N/K)
    if debug:
        print("Number of players ", N)
        print("Number of clusters ", K)
        print("Number of players per Cluster ", playersPerCluster)
    decisions = np.full(R, np.nan)
    historicL0 = np.full((N, histlenL0), np.nan) # Empty history matrix (N x histlen) or (number of playes x History Len)
    historicL1 = np.full((K, histlenL1), np.nan)
    utilities = np.zeros((N, R)) # Empty utility matrix (N x R)
    falsenegativesL0 = np.zeros((N, R)) # Empty utility matrix (N x R)
    falsenegativesL1 = np.zeros((K, R)) # Empty utility matrix (K x R)
    falsenegativesBoth = np.zeros((N, R)) # Empty utility matrix (N x R)
    for i in range(R):
        # Roll historic to the left
        if (i > histlenL0):
            historicL0 = np.roll(historicL0, -1, axis=1)
            
        theta = np.zeros(N)
        # copy declared values at the end of the historic
        historicL0[:, min(i, histlenL0 - 1)] = declaredVals[: , i]

        for j in range(N):
            if debug: 
                print ("player ", j, " has values ", historicL0[j])
            if KSTest and stats.kstest(historicL0[j, 0:min(i+1, histlenL0)], 'uniform').pvalue < (1 - maxpvalL0):
                if debug: print("False negative")
                theta[j] = np.random.uniform(0, 1, 1)
                falsenegativesL0[j, i] = 1
            else:
                theta[j] = declaredVals[j, i]
                
        # copy declared values at the end of the historic
        historicL0[:, min(i, histlenL0 - 1)] = theta
        
        thetaUp = np.zeros(K)
        decisionsL1 = np.full(K, np.nan)
        for k in range(K):     
            valsCluster = theta[(k)*playersPerCluster:(k+1)*playersPerCluster]
            decisionsL1[k] = int(np.argmin(valsCluster))
            player = int((k)*playersPerCluster + decisionsL1[k])
            if debug: 
                print ("cluster ", k, " has values ", valsCluster)
                print ("cluster ", k, " has player ", player)
                print ("cluster ", k, " has value ", valsCluster[int(decisionsL1[k])])
                print ("cluster ", k, " has range ", (k, min(i, histlenL1 - 1)))
            historicL1[k, min(i, histlenL1 - 1)] = valsCluster[int(decisionsL1[k])]
            if KSTest and stats.kstest(historicL1[k, 0:min(i+1, histlenL1)], stats.beta(a=1., b=playersPerCluster).cdf).pvalue < (1 - maxpvalL1):
                if debug: print("False negative")
                thetaUp[k] = np.random.uniform(0, 1, 1)
                falsenegativesL1[k, i] = 1
                falsenegativesBoth[player, i] = 1
            else:
                thetaUp[k] = valsCluster[int(decisionsL1[k])]
                
        #print(thetaUp)        
        dCluster = np.argmin(thetaUp)
        d = int(dCluster*playersPerCluster + decisionsL1[dCluster])
        if debug: print("Win cluster ", dCluster, " and player ", d)
        decisions[i] = d
        utilities[d, i] = trueVals[d, i]
    return decisions, utilities, falsenegativesL0, falsenegativesL1, falsenegativesBoth